In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

2024-12-11 12:34:08.291941: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-11 12:34:08.337947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733900648.389051  197351 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733900648.405132  197351 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 12:34:08.450471: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is enabled.")
    except RuntimeError as e:
        print(e)
else:
    num_cpus = os.cpu_count()
    tf.config.threading.set_intra_op_parallelism_threads(num_cpus)
    tf.config.threading.set_inter_op_parallelism_threads(num_cpus)

2024-12-11 12:34:13.723704: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
df_train = pd.read_csv('../input/credit-dset/clean_trained.csv').drop(['Number'],axis = 1)
df_test = pd.read_csv('../input/credit-dset/test_cleaned.csv').drop(['Number'],axis = 1)
print(df_train.info(),end='\n\n')
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     80000 non-null  object 
 1   Age                       80000 non-null  int64  
 2   Profession                80000 non-null  object 
 3   Income_Annual             80000 non-null  float64
 4   Base_Salary_PerMonth      80000 non-null  float64
 5   Total_Bank_Accounts       80000 non-null  int64  
 6   Total_Credit_Cards        80000 non-null  int64  
 7   Rate_Of_Interest          80000 non-null  int64  
 8   Total_Current_Loans       80000 non-null  int64  
 9   Delay_from_due_date       80000 non-null  int64  
 10  Total_Delayed_Payments    80000 non-null  float64
 11  Credit_Limit              80000 non-null  float64
 12  Total_Credit_Enquiries    80000 non-null  float64
 13  Credit_Mix                80000 non-null  object 
 14  Curren

In [4]:
one_hot_encode_cols = ['Month','Profession']
for i in one_hot_encode_cols:
    ohe = OneHotEncoder(sparse_output=False)
    encoded_train = ohe.fit_transform(df_train[[i]])
    encoded_test = ohe.transform(df_test[[i]])
    encoded_cols = [f"{i}_{j}" for j in ohe.categories_[0]]
    df_train_encoded = pd.DataFrame(encoded_train, columns=encoded_cols)
    df_test_encoded = pd.DataFrame(encoded_test, columns=encoded_cols)
    df_train = pd.concat([df_train, df_train_encoded], axis=1).drop(columns=[i])
    df_test = pd.concat([df_test, df_test_encoded], axis=1).drop(columns=[i])

In [5]:
cred_mix_dict = {"Bad" : 0,"Standard" : 1,"Good" : 2}
df_train['Credit_Mix'] = df_train['Credit_Mix'].map(cred_mix_dict)
df_test['Credit_Mix'] = df_test['Credit_Mix'].map(cred_mix_dict)

In [6]:
df_train[['pay_type','val_pay']] = df_train['Payment_Behaviour'].str.split(pat = '_spent_',n=1,expand = True)
df_test[['pay_type','val_pay']] = df_test['Payment_Behaviour'].str.split(pat = '_spent_',n=1,expand = True)
df_train = df_train.drop(['Payment_Behaviour'],axis=1)
df_test = df_test.drop(['Payment_Behaviour'],axis=1)
pay_type_dict = {'Low' : 0, 'High' : 1}
val_pay_dict = {'Small_value_payments' : 0,'Medium_value_payments' : 1,'Large_value_payments' : 2}
df_train['pay_type'] = df_train['pay_type'].map(pay_type_dict)
df_test['pay_type'] = df_test['pay_type'].map(pay_type_dict)
df_train['val_pay'] = df_train['val_pay'].map(val_pay_dict)
df_test['val_pay'] = df_test['val_pay'].map(val_pay_dict)

In [7]:
yes_no_dict = {'Yes' : 1,'No' : 0}
df_train['Payment_of_Min_Amount'] = df_train['Payment_of_Min_Amount'].map(yes_no_dict)
df_test['Payment_of_Min_Amount'] = df_test['Payment_of_Min_Amount'].map(yes_no_dict)

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       80000 non-null  int64  
 1   Income_Annual             80000 non-null  float64
 2   Base_Salary_PerMonth      80000 non-null  float64
 3   Total_Bank_Accounts       80000 non-null  int64  
 4   Total_Credit_Cards        80000 non-null  int64  
 5   Rate_Of_Interest          80000 non-null  int64  
 6   Total_Current_Loans       80000 non-null  int64  
 7   Delay_from_due_date       80000 non-null  int64  
 8   Total_Delayed_Payments    80000 non-null  float64
 9   Credit_Limit              80000 non-null  float64
 10  Total_Credit_Enquiries    80000 non-null  float64
 11  Credit_Mix                80000 non-null  int64  
 12  Current_Debt_Outstanding  80000 non-null  float64
 13  Ratio_Credit_Utilization  80000 non-null  float64
 14  Credit

In [9]:
encoder_final = OrdinalEncoder(categories=[['Poor', 'Standard', 'Good']])
df_train['Credit_Score'] = encoder_final.fit_transform(df_train[['Credit_Score']])
df_train['Credit_Score'].unique()

array([1., 0., 2.])

In [10]:
X = df_train.drop(['Credit_Score'],axis=1)
Y = df_train['Credit_Score']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [11]:
X_reshaped = X_scaled.reshape(-1, X_scaled.shape[1], 1)
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, Y, test_size=0.2, random_state=42)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')  # Assuming 3 target classes
])
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
    
#     tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling1D(pool_size=2),
    
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dropout(0.3),
#     tf.keras.layers.Dense(3, activation='softmax')  # Assuming 3 target classes
# ])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/home/abhinav/Downloads/venv_ml/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=64)

Epoch 1/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.6122 - loss: 0.8436 - val_accuracy: 0.6682 - val_loss: 0.7027
Epoch 2/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.6624 - loss: 0.7155 - val_accuracy: 0.6692 - val_loss: 0.7019
Epoch 3/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.6715 - loss: 0.6993 - val_accuracy: 0.6711 - val_loss: 0.6939
Epoch 4/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.6727 - loss: 0.6972 - val_accuracy: 0.6728 - val_loss: 0.6876
Epoch 5/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - accuracy: 0.6803 - loss: 0.6913 - val_accuracy: 0.6768 - val_loss: 0.6875
Epoch 6/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - accuracy: 0.6803 - loss: 0.6876 - val_accuracy: 0.6725 - val_loss: 0.6895
Epoch 7/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - accuracy: 0.6755 - loss: 0.6885 - val_accuracy: 0.6773 - val_loss: 0.6859
Epoch 8/100
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 17s 17ms/step - accuracy: 

In [14]:
test_predictions = model.predict(scaler.transform(df_test.drop(['ID'],axis=1)))
pred_class = test_predictions.argmax(axis=1)
pd.DataFrame({
    'ID' : df_test['ID'],
    'Predicted' : encoder_final.inverse_transform(pred_class.reshape(-1,1)).reshape(-1)
}).to_csv('sub.csv',index=False)

625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
